In [1]:
from crimm.Modeller import TopologyGenerator
from crimm.Modeller.SeqChainGenerator import SeqChainGenerator
from crimm.Modeller.TopoFixer import fix_chain
from crimm.Visualization import View, show_nglview_multiple

We define a function to show licorice since the default rendering is "cartoon" and does not show the side chains.

In [2]:
def show_licorice(chain):
    view = View()
    view.load_entity(chain)
    view.clear_representations()
    view.add_representation('licorice')
    return view

def show_licorice_multiple(chains):
    view = View()
    for chain in chains:
        component = view.load_entity(chain)[0]
        view.clear_representations(component = component._index)
        view.add_representation('licorice', component = component._index)
    return view

## Generate a Chain Object from Sequence
The `SeqChainGenerator` class is used to construct RNA and polypeptide chain from sequences. The chain will be constructed from the *internal coordinate* (ic) definitions from **CHARMM36** residue topology files (rtf).

In [3]:
generator = SeqChainGenerator()

### Chain from One-letter Sequences for a Short RNA

The `chain_type` parameter is needed to identify the correct rtf. And of course, protein one-letter sequence is also accepted

In [4]:
# chain_type is not case sensitive. RNA or polyribonucleotide can also be accepted
generator.set_sequence('CUCGAAUCA', chain_type = 'rna') 
rna_chain = generator.create_chain()
rna_chain

NGLWidget()

<Polyribonucleotide id=A Residues=9>


In [5]:
show_licorice(rna_chain)

View()

### Chain from Three-letter Sequece for ADA Tripeptide

The residue names in the three-letter sequences need to be separated by white spaces.

In [6]:
# Protein or Polypeptide(L) is also accepted
# Since we will patch the chain to make ALA ADA ALA tripeptide, we start with only two residues here
generator.set_three_letter_sequence('ASP ALA', chain_type = 'polypeptide')
peptide_chain = generator.create_chain()
peptide_chain

NGLWidget()

<Polypeptide(L) id=A Residues=2>


In [7]:
show_licorice(peptide_chain)

View()

## Minimize the Structure

We can load the constructed chain into pyCHARMM by `load_chain` function in `crimm.Adaptors.pyCHARMMAdaptors`. Once it's loaded, all pyCHARMM functions can be called and applied to the loaded chain, just like it was loaded from pdb and psf files.

In [8]:
from crimm.Adaptors.pyCHARMMAdaptors import load_chain, load_topology
from crimm.Adaptors.pyCHARMMAdaptors import minimize, sync_coords, empty_charmm
from pycharmm.psf import delete_atoms, get_natom
from pycharmm.settings import set_verbosity

## Generate the Topology and Apply Parameters from crimm

Before we can load the chain, the **topology** and **parameter** definitions need to be generated on the chain we just constructed. Currently, we still need to specify the `chain_type` for our chain, but it will become automated soon. The generation routine will be simplified very soon.

Here, we make a copy of the original chain for the structure comparison purposes

In [9]:
minimized_chain = peptide_chain.copy()

Currently, default terminal patches have not been implemented (since they need to be looked up from rtf files for different residues). We still need to specify patches when we generate the topology

In [10]:
topo = TopologyGenerator()
topo.generate(minimized_chain, first_patch='ACE', last_patch='CT3')

/home/truman/crimm/crimm/Modeller/TopoLoader.py:537: UserWarning: No cmap found in <Polypeptide(L) id=A Residues=2>.
  warnings.warn(


<TopologyElementContainer for <Polypeptide(L) id=A Residues=2> with bonds=33, angles=57, dihedrals=72, impropers=7, cmap=0>

Here, again we need to fill the ic table, since we just generated new residue definition with terminal patches, and these patch definitions' ic need to be filled from parameter files.

In [11]:
fixed_atoms = fix_chain(minimized_chain)

In [12]:
minimized_chain

NGLWidget()

<Polypeptide(L) id=A Residues=2>


### Load Parameter and Topology Files
Before we can load the chain, the correct rtf and parameter files have to be loaded. We have utility function `load_topology` and `load_parameters` in the `pyCHARMMAdaptors` to help load the rtf and prm files correctly.

In [13]:
load_topology(topo)

  
 CHARMM>     read rtf card -
 CHARMM>     name /tmp/tmpeazqn18j
 VOPEN> Attempting to open::/tmp/tmpeazqn18j::
 MAINIO> Residue topology file being read from unit  91.
 TITLE> *RTF LOADED FROM CRIMM
 TITLE> 36  2
 VCLOSE: Closing unit   91 with status "KEEP"
  
 CHARMM>     
  
  
 CHARMM>     read param card -
 CHARMM>     name /tmp/tmprm2pijvc -
 CHARMM>     flex
 VOPEN> Attempting to open::/tmp/tmprm2pijvc::

          PARAMETER FILE BEING READ FROM UNIT 91
 TITLE> *PRM LOADED FROM CRIMM
 TITLE> *>>>> CHARMM36 ALL-HYDROGEN PARAMETER FILE FOR PROTEINS <<<<<<<<<<
 TITLE> *>>>>> INCLUDES PHI, PSI CROSS TERM MAP (CMAP) CORRECTION <<<<<<<<
 TITLE> *>>>>>>>>>>>>>>>>>>>>>>>>>> JAN. 2016 <<<<<<<<<<<<<<<<<<<<<<<<<<<<
 TITLE> * ALL COMMENTS TO THE CHARMM WEB SITE: WWW.CHARMM.ORG
 TITLE> *             PARAMETER SET DISCUSSION FORUM
 TITLE> *
 PARMIO> NONBOND, HBOND lists and IMAGE atoms cleared.
 VCLOSE: Closing unit   91 with status "KEEP"
  
 CHARMM>     
  


A wrapper function to delete all atoms in pyCHARMM. If there is no atom exists, an error will be raised.

In [14]:
empty_charmm() # if there is anything in charmm, delete it
load_chain(minimized_chain)

  
 CHARMM>     read sequence pdb -
 CHARMM>     name /tmp/tmp4agrceav
 VOPEN> Attempting to open::/tmp/tmp4agrceav::
 MAINIO> Sequence information being read from unit  91.
 TITLE>  *

          RESIDUE SEQUENCE --     2 RESIDUES
          ASP ALA 
 ***** Message from SEQRDR ***** THE SYSTEM CONTAINS  1 TITRATABLE GROUPS
 THE USER MUST PREDETERMINE THE PROTONATION STATE THROUGH THE SEQUENCE AND RTF
 HIS -  0  HSD -  0  HSE -  0  HSP -  0  ASP -  1  GLU -  0  LYS -  0  TYR -  0
 VCLOSE: Closing unit   91 with status "KEEP"
  
 CHARMM>     
  
 THE PATCH 'ACE' WILL BE USED FOR THE FIRST RESIDUE
 THE PATCH 'CT3' WILL BE USED FOR THE LAST  RESIDUE
 AUTGEN: Autogenerating specified angles and dihedrals.
 GENPSF> Segment   1 has been generated. Its identifier is PROA.
 PSFSUM> PSF modified: NONBOND lists and IMAGE atoms cleared.
 PSFSUM> Summary of the structure file counters :
         Number of segments      =        1   Number of residues   =        2
         Number of atoms         =  

There is a wrapper function for steepest descend and adopted basis Newton-Raphson minimization method.

In [15]:
# set verbosity to zero to reduce output from minimization steps
prev_level = set_verbosity(0)
minimize(sd_nstep=200, abnr_nstep=0)
set_verbosity(prev_level)

Adopted Basis Newton-Raphson minimization not performed


0

## Synchronize the Structure with pyCHARMM
Since the coordinates are minimized in pyCHARMM, we need to update the coords in our chain object by calling `sync_coords` function. This need to be called whenenver a computation is done and you want to update the structure on crimm side. 

**Note**: if there is any structure descripency between the pyCHARMM and crimm chain, the synchronization will be aborted. This is likely due to modification of the crimm chain after it was loaded, or a wrong chain is supplied as the function argument.

In [16]:
sync_coords(minimized_chain)

Synchronized: <Polypeptide(L) id=A Residues=2>


In [17]:
minimized_chain

NGLWidget()

<Polypeptide(L) id=A Residues=2>


### Compare the Minimized Structure and the Original

In [18]:
show_licorice_multiple([minimized_chain, peptide_chain])

View()